In [1]:
import os
from os import listdir
import time
import h5py as h5
import numpy as np
from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr
import pandas
import scipy
from scipy import interpolate
from scipy.interpolate import griddata


In [2]:
#execution time
start_time = time.time()

# Define Input data path and output file path
path = 'D:/GIC_2020/01_GCOM_C/002_Data/04_timeseriesanalysis/ARPL/data/'
outpath = 'D:/GIC_2020/01_GCOM_C/002_Data/04_timeseriesanalysis/results/ARPL_AOT_1D_Feb/ARPL_AOT_1D_Feb_CM/'

#choose product name
product_name = "AROT_pol_land"

In [3]:
#Define resolution of the raster
resolution = 0.008333334

def open_img_data(img_path,flag_num,h_tile, v_tile,img_nlines,img_npixels,reso):
    f = h5.File(img_path, "r")
    dset = f['/Image_data/'+ product_name]
    ARPL_DN_data = dset[:]
    #Take imge data calculated with slope and offset data
    invalid_val = np.NaN
    slope = dset.attrs['Slope'][0]
    offset = dset.attrs['Offset'][0]
    err_dn = dset.attrs['Error_DN'][0]
    min_dn = dset.attrs['Minimum_valid_DN'][0]
    max_dn = dset.attrs['Maximum_valid_DN'][0]
    PR_data = ARPL_DN_data * slope + offset
    PR_data[ARPL_DN_data == err_dn] = invalid_val
    PR_data[(ARPL_DN_data < min_dn) | ( ARPL_DN_data > max_dn)] = invalid_val
    #Take cloud masked image data
    flag_val = np.sum(np.power(2, np.array(flag_num, dtype=np.uint32)))
    qa_flag_dataset_name = [name for name in f['/Image_data'].keys() if 'QA_flag'in name][0]
    qa_flag_data = f['Image_data/' + qa_flag_dataset_name][:]
    qa_flag_data = np.bitwise_and(qa_flag_data, flag_val).astype(np.bool)

    New_PR_data = np.zeros((img_nlines, img_npixels))
    for r in range(0,img_nlines):
        for y in range(0,img_npixels):
            if qa_flag_data[r][y] ==True:
                New_PR_data[r][y] = float("nan")
            else:
                New_PR_data[r][y] = PR_data[r][y]
    
    #getting long,lat data            
    u, v = np.meshgrid(np.arange(0, img_npixels), np.arange(0, img_nlines))
    lat = 90. - (v_tile * img_nlines + v + 0.5) * reso
    lon = ((h_tile * img_npixels + u + 0.5) * reso - 180.) / np.cos(np.deg2rad(lat))
    lon[(lon < -180) | (lon > 180)] = np.NaN
    lat[np.isnan(lon)] = np.NaN

    return New_PR_data,lon,lat





In [4]:
#Getting the data list
list = os.listdir(path)

for i in range(0,len(list)):
    
    file_name = (str(list[i]).split('.'))[0]
    PR_data,longitude,latitude= open_img_data(path + str(list[i]) ,2,27,7,1200,1200,resolution)

    lati=np.ravel(latitude)
    long=np.ravel(longitude)
    Data=np.ravel(PR_data)
    Data[np.isnan(Data)] = float("nan")
    df = pandas.DataFrame(
        { 'Data':Data,
          'Lon': long,
          'Lat': lati,
             })
    arrD = np.asarray(df.Data)
    arrlon = np.asarray(df.Lon)
    arrlat = np.asarray(df.Lat)

    #Interpolating the data to a grid (linear)
    from scipy.interpolate import griddata
    grid_yy = np.arange(arrlat.min(),arrlat.max(),resolution)
    grid_xx = np.arange(arrlon.min(),arrlon.max(),resolution)

    grid_x,grid_y = np.meshgrid(grid_xx, grid_yy)
    g=grid_y[::-1]
    grid_z0 = griddata((arrlon[:],arrlat[:]), Data, (grid_x,g), method='linear', fill_value= float("nan"))

    #write Grid data into a raster
    image_data = grid_z0
    nrows = int(((arrlat.max()-arrlat.min())/resolution)+1)
    ncols = int(((arrlon.max()-arrlon.min())/resolution)+1)

    geotransform=([arrlon.min(),resolution,0,arrlat.max(),0, -resolution])   
    # That's (top left x, w-e pixel resolution, rotation (0 if North is up), top left y, rotation (0 if North is up), n-s pixel resolution)


    output_raster = gdal.GetDriverByName('GTiff').Create(outpath+ file_name +'.tif',ncols, nrows, 1 ,gdal.GDT_Float64)  # Open the file
    output_raster.SetGeoTransform(geotransform)  
    srs = osr.SpatialReference()                
    srs.ImportFromEPSG(4326)                                                                                                         
    output_raster.SetProjection( srs.ExportToWkt() )  
    output_raster.GetRasterBand(1).WriteArray(image_data) 
    output_raster.FlushCache()

In [5]:
print("--- %s seconds ---" % (time.time() - start_time))


--- 959.4345183372498 seconds ---
